# RAG Fashion Chatbot — Project Overview

### What this project does

A retrieval-augmented chatbot that answers **store FAQs** and **product queries** by semantically searching Weaviate (with `text2vec-transformers`) and then composing grounded answers with an LLM (Together), returning product IDs and concise explanations.


## Mental map — end-to-end pipeline 

```
User Query
   │
   ▼
answer_query(query, model)                          ←  (entry point)
   │
   ├─ check_if_faq_or_product(query)                ←  "FAQ" / "Product"
   │
   ├─ if FAQ:
   │     query_on_faq(query)                        ← 
   │        ├─ faq_collection.query.near_text(...)      (top-5)
   │        ├─ generate_faq_layout(results)            ← (format FAQ ctx)
   │        └─ generate_params_dict(PROMPT, ...)        (kwargs for LLM)
   │
   └─ if Product:
         query_on_products(query)                   ← 
            ├─ decide_task_nature(query)                ←  ("creative"/"technical")
            ├─ get_params_for_task(label)               ←  (decoding presets)
            ├─ get_relevant_products_from_query(query)  ←  (near_text top-20)
            │     └─ products_collection.query.near_text(...)
            │     ├─ generate_metadata_from_query(query)   ←  (LLM → JSON)
            │    
            │       
            ├─ generate_items_context(results)         ←  (format product ctx)
            └─ generate_params_dict(PROMPT, role='assistant', **presets)
               (then call generate_with_single_input(**kwargs) to get the final answer)
```

**Why this structure?**

* **Classify early** → route to the right knowledge (FAQ vs Product).
* **Retrieve first** → ground the LLM on real items/answers, reduce hallucinations.
* **Decoding presets** → creative vs technical tone control.









In [285]:
import os
import json
import requests

import weaviate
from weaviate.classes.init import Timeout
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import MetadataQuery
from weaviate.classes.query import Filter

import pandas as pd
import joblib

In [286]:
# Utilities
from utils import (
    generate_with_single_input,
)

### Connect to local Weaviate

Creates a client to the Dockerized Weaviate instance at `http://localhost:8080` (gRPC `50051`). 


In [287]:
# Connect to local Weaviate from Docker compose
client = weaviate.connect_to_local(
    host="localhost",
    port=8080,
    grpc_port=50051,
    skip_init_checks=True,  # tolerate /ready=503
)

client.is_ready()


True

### Datasets:

- Product: Contains the products and their information.
- FAQ: Contains the FAQ data.

#### Products Data

In [288]:
# Loading products data
# load it as a list of JSON files first.
products_data = joblib.load('dataset/clothes_json.joblib')

In [289]:
# Let's get one example
products_data[0]

{'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Shirts',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011,
 'usage': 'Casual',
 'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
 'price': 67.0,
 'product_id': 15970}

#### FAQ Data
Each entry is a dictionary containing the following keys: `question`, `answer`, and `type`.

In [290]:
faq = joblib.load("dataset/faq.joblib")

In [291]:
# Get an example
faq[:2]

[{'question': 'What are your store hours?',
  'answer': 'Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.',
  'type': 'general information'},
 {'question': 'Where is Fashion Forward Hub located?',
  'answer': 'Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State.',
  'type': 'general information'}]

### LLM call Func (Together) + test


**What it does**

* Builds one `payload` in OpenAI chat format:
  `{"model": ..., "messages": [{"role": role, "content": prompt}], ...}`
* Calls the first available provider in this order:

  1. **Together** if `together_api_key` arg is passed
  2. **Together** if `TOGETHER_API_KEY` env var is set
  3. **OpenAI** if `OPENAI_API_KEY` env var is set
  4. Otherwise raises a “No API key found” error
* Returns a dict (from `.model_dump()`), so downstream parsing is the same regardless of provider.

**Inputs you’ll care about**

* `prompt` (str): user prompt placed into `messages=[{"role": role, "content": prompt}]`.
* `role` (str): usually `"user"`; you can pass `"system"` if you want the whole prompt as a system message.
* `model` (str): default `"meta-llama/Llama-3.2-3B-Instruct-Turbo"` for Together; OpenAI fallback uses `"gpt-4o-mini"`.
* `max_tokens`, `temperature`, `top_p`: standard decoding controls.
* `**kwargs`: forwarded to the provider call (use sparingly; keep OpenAI/Together compatibility in mind).

**Environment variables**

* `TOGETHER_API_KEY` → tries Together first.
* `OPENAI_API_KEY` → used only if Together is not available.

**Return shape (OpenAI-style)**

* `result["choices"][0]["message"]["content"]` → assistant text
* `result["usage"]["total_tokens"]` (if provided by the SDK)
* `result["model"]` → model identifier actually used


**Smoke test at the bottom**

* Calls the function with a short prompt and pretty-prints the JSON so you can confirm the provider, `choices`, and `usage` fields look right.


In [292]:
# The output is a dictionary containing the role and content from the LLM call, as well as the token usage.:
result = generate_with_single_input("What are the primary colors?")
print(json.dumps(result, indent = 2))

{
  "id": "oE37omS-57nCBj-98a19b24edeecb0d",
  "object": "chat.completion",
  "created": 1759716455,
  "model": "meta-llama/Llama-3.2-3B-Instruct-Turbo",
  "choices": [
    {
      "index": 0,
      "logprobs": null,
      "seed": 15704831269139120000,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "The primary colors are:\n\n1. Red\n2. Blue\n3. Yellow\n\nThese colors cannot be created by mixing other colors together, and they are the base colors used to create all other colors.",
        "tool_calls": []
      }
    }
  ],
  "prompt": [],
  "usage": {
    "prompt_tokens": 41,
    "completion_tokens": 42,
    "total_tokens": 83,
    "cached_tokens": 0
  }
}


#### to retreive just the content

In [293]:
# Retreive just the content
print(result['choices'][0]['message']['content'])

The primary colors are:

1. Red
2. Blue
3. Yellow

These colors cannot be created by mixing other colors together, and they are the base colors used to create all other colors.


#### To retrieve the total number of tokens

In [294]:
# The total tokens count (input + output) for this is:
print(result['usage']['total_tokens'])

83


#### Function to generate the parameters dictionary



In [295]:
def generate_params_dict(
    prompt: str,
    temperature: float = 1.0,
    role: str = 'user',
    top_p: float = 1.0,
    max_tokens: int = 500,
    model: str = "meta-llama/Llama-3.2-3B-Instruct-Turbo"
) -> dict:
    """
    Generates a dictionary of parameters for calling a Language Learning Model (LLM),

    Returns:
        dict: A dictionary containing all specified parameters which can then be used to configure and execute a call to the LLM.
    """

    kwargs = {
        "prompt": prompt,
        "role": role,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "model": model
    }
    return kwargs

In [296]:
kwargs = generate_params_dict("Solve 3x^2 + 5 = 0")
print(kwargs)

{'prompt': 'Solve 3x^2 + 5 = 0', 'role': 'user', 'temperature': 1.0, 'top_p': 1.0, 'max_tokens': 500, 'model': 'meta-llama/Llama-3.2-3B-Instruct-Turbo'}


#### TEST

In [297]:
# call the LLM 
result = generate_with_single_input(**kwargs)
content = result['choices'][0]['message']['content']
total_tokens = result['usage']['total_tokens']
print(f"Content: {content}\n\nTotal Tokens: {total_tokens}")

Content: To solve the equation 3x^2 + 5 = 0, we need to isolate x.

First, subtract 5 from both sides:

3x^2 = -5

Next, divide both sides by 3:

x^2 = -5/3

Since the square of any real number cannot be negative, there are no real solutions to the equation. However, we can express the complex solutions as:

x = ±i√(5/3)

where i is the imaginary unit (i = √(-1)).

Total Tokens: 159


### Classify query as **FAQ** vs **Product**

Uses a short LLM prompt (low temperature, small max_tokens) to label an input `query` as either **FAQ** (store policies, contact, hours, etc.) or **Product** (item-specific info, recommendations, filters). 
Returns the normalized label (`"FAQ"` / `"Product"` or `"undefined"`) plus the model’s `total_tokens`.


In [298]:
def check_if_faq_or_product(query):
    """
    Determines whether a given instruction prompt is related to a frequently asked question (FAQ) or a product inquiry.

    Parameters:
    - query (str): The instruction or query that needs to be labeled as either FAQ or Product related.
    - simplified (bool): If True, uses a simplified prompt.

    Returns:
    - str: The label 'FAQ' if the prompt is deemed a frequently asked question, 'Product' if it is related to product information, or
      None if the label is inconclusive.
    """
 

    PROMPT = f"""Label the following instruction as an FAQ related answer or a product related answer for a clothing store.
    Product related answers are answers specific about product information or that needs to use the products to give an answer.
    Examples:
            Is there a refund for incorrectly bought clothes? Label: FAQ
            Where are your stores located?: Label: FAQ
            Tell me about the cheapest T-shirts that you have. Label: Product
            Do you have blue T-shirts under 100 dollars? Label: Product
            What are the available sizes for the t-shirts? Label: FAQ
            How can I contact you via phone? Label: FAQ
            How can I find the promotions? Label: FAQ
            Give me ideas for a sunny look. Label: Product
    Return only one of the two labels: FAQ or Product, nothing more.
    Query to classify: {query}
                """
        
    # Get the kwargs dictinary to call the llm, with PROMPT as prompt, low temperature (0 or near 0) and max_tokens = 10
    kwargs = generate_params_dict(PROMPT, temperature = 0, max_tokens = 10)

    response = generate_with_single_input(**kwargs) 
    label = response['choices'][0]['message']['content']
    total_tokens = response['usage']['total_tokens']

    if 'faq' in label.lower():
        label = 'FAQ'
    elif 'product' in label.lower():
        label = 'Product'
    else:
        label = 'undefined'

    return label, total_tokens

### Format a list of FAQs into a prompt-ready block

Takes a list of FAQ dicts (`{'question','answer','type'}`) and builds a single newline-separated string like `Question: … Answer: … Type: …` per item. Useful for stuffing the most relevant FAQs into an LLM prompt in a compact, readable form.


In [299]:
def generate_faq_layout(faq_dict):
    """
    Generates a formatted string layout for a list of FAQs.

    This function iterates through a dictionary of frequently asked questions (FAQs) and constructs
    a string where each question is followed by its corresponding answer and type.

    Parameters:
    - faq_dict (list): A list of dictionaries, each containing keys 'question', 'answer', and 'type' 
      representing an FAQ entry.

    Returns:
    - str: A string representing the formatted layout of FAQs, with each entry on a separate line.
    """
    # Initialize an empty string
    t = ""
    
    # Iterate over every FAQ question in the FAQ list
    for f in faq_dict:
        # Append the question with formatted string (remember to use f-string and access the values as f['question'], f['answer'] and so on)
        # Also, do not forget to add a new line character (\n) at the end of each line.
        t += f"Question: {f['question']} Answer: {f['answer']} Type: {f['type']}\n" 

    return t

In [300]:
# You can generate a full faq_layout with the entire FAQ questions
faq_layout = generate_faq_layout(faq)
print(faq_layout[:1000])

Question: What are your store hours? Answer: Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday. Type: general information
Question: Where is Fashion Forward Hub located? Answer: Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State. Type: general information
Question: Do you have a physical store location? Answer: At this time, we operate exclusively online. This allows us to offer a broader selection and lower prices directly to you. Type: general information
Question: How can I create an account with Fashion Forward Hub? Answer: Click on 'Sign Up' in the top right corner of our website and follow the instructions to set up your account. Type: general information
Question: How do I subscribe to your newsletter? Answer: To receive the latest updates and promotions, sign up for our newsletter at the bottom of our homepage. Type: general information
Question:

### Vector DB: Create **Faq** collection (vectorized) and batch-insert FAQs

Drops any existing `Faq` collection, recreates it with the `text2vec-transformers` vectorizer (enabling `near_text` search), defines `question/answer/type` properties, then batch-inserts all FAQ items using stable UUIDs derived from each question.


In [301]:
from weaviate.util import generate_uuid5

try:
    if client.collections.exists("Faq"):
        client.collections.delete("Faq")
except Exception as e:
    print("Warning deleting Faq:", e)

client.collections.create(
    name="Faq",
    vectorizer_config=Configure.Vectorizer.text2vec_transformers(),  
    properties=[
        Property(name="question", data_type=DataType.TEXT),
        Property(name="answer",   data_type=DataType.TEXT),
        Property(name="type",     data_type=DataType.TEXT),
    ],
)

In [302]:
# Insert FAQ docs
faq_collection = client.collections.get("Faq")
with faq_collection.batch.fixed_size(batch_size=20, concurrent_requests=5) as batch:
    for document in faq:  # [{'question':..., 'answer':..., 'type':...}, ...]
        uuid = generate_uuid5(document['question'])
        batch.add_object(properties=document, uuid=uuid)

### Semantic search over FAQs

Runs a `near_text` query against the vectorized `Faq` collection to fetch the top 5 relevant Q&As for “What is the return policy?”, then prints `question -> answer`. (Assumes `Faq` is created with `text2vec-transformers` and populated.)


In [303]:
# Semantic search
res = faq_collection.query.near_text(query="What is the return policy?", limit=5)
for obj in res.objects:
    print(obj.properties["question"], "->", obj.properties["answer"])

What is your return policy timeframe? -> We accept returns within 30 days of delivery. Conditions apply for specific categories like accessories.
How long does it take to process a return? -> Return processing typically takes 5-7 business days from when the item is received at our warehouse.
Are return shipping costs covered? -> We provide a prepaid return label for domestic returns. For international returns, shipping is at the customer's cost.
Can I return a sale item? -> Sale items are final sale and cannot be returned or exchanged, unless stated otherwise.
How do I exchange an item? -> Initiate an exchange through our Returns Center, selecting the item you wish to exchange and the desired replacement.


### Build prompt from top-5 semantic FAQ hits

Looks up the 5 most relevant FAQ entries via `near_text`, reverses them (least → most relevant at the bottom) to bias recency in the prompt, renders a compact FAQ layout, and returns **LLM call parameters** (`kwargs`) using `generate_params_dict`.


In [304]:
def query_on_faq(query, **kwargs):
    """
    Constructs a prompt to query an FAQ system and generates a response.

    This function integrates an FAQ layout into the prompt to help generate a suitable answer to the given query
    using a language model. It uses semantic search to extract a relevant subset of FAQ questions

    Parameters:
    - query (str): The query about which the function seeks to provide an answer from the FAQ.
    - **kwargs: Optional keyword arguments for extra configuration of prompt parameters.

    Returns:
    - str: The response generated from the language model based on the input query and FAQ layout.

    """    

    # Get the 5 most relevant FAQ objects, in this case limit = None
    results = faq_collection.query.near_text(f"{query}", limit = 5)    

    # Transform the results in a list of dictionary
    results = [x.properties for x in results.objects] 
    # Reverse the order to add the most relevant objects in the bottom, so it gets closer to the end of the input
    results.reverse() 
    # Generate the faq layout with the new list of FAQ questions `results`
    faq_layout = generate_faq_layout(results) 

    # Different prompt to deal with this new scenario. 
    PROMPT = (f"You will be provided with a query for a clothing store regarding FAQ. It will be provided relevant FAQ from the clothing store." 
    f"Answer the query based on the relevant FAQ provided. They are ordered in decreasing relevance, so the first is the most relevant FAQ and the last is the least relevant."  
    f"Answer the instruction based on them. You might use more than one question and answer to make your answer. Only answer the question and do not mention that you have access to a FAQ.\n"  
    f"<FAQ>\n"  
    f"RELEVANT FAQ ITEMS:\n{faq_layout}\n"  
    f"</FAQ>\n" 
    f"Query: {query}")   
 
    
    # Generate the parameters dict with PROMPT and **kwargs 
    kwargs = generate_params_dict(PROMPT, **kwargs) 

    return kwargs

In [305]:
# Get the dictionary of arguments
kwargs = query_on_faq("I received the dress I ordered but I don't like it. How can I return it?")

In [306]:
# The number of split tokens in this prompt is:
print(len(kwargs['prompt'].split()))

254


#### TEST 

In [307]:
# Run the inference
result = generate_with_single_input(**kwargs)

In [308]:
print(result['choices'][0]['message']['content'])

To return your dress, you can initiate the return process through our Returns Center. Select the item you wish to return, and then select the desired replacement you would like to exchange it for. Once you initiate the return, please allow 5-7 business days for return processing. 

Additionally, you should note that it is generally not possible to return a sale item, as they are specified as final sale and cannot be returned or exchanged unless the specific policy allows for it. However, in your case, since the item is not a sale item, you should be able to return it for a refund or exchange.


In [309]:
# Get the total tokens
print(result['usage']['total_tokens'])

474


### Classify query as **creative** vs **technical**

Prompts the LLM (low temperature, `max_tokens=1`) to label a user query as **creative** (suggestions, styling ideas) or **technical** (product details, availability, prices). Returns the raw label text from the model and `total_tokens` for telemetry.


In [310]:
def decide_task_nature(query):
    """
    Determines the nature of a query, labeling it as either creative or technical.
    This function constructs a prompt for a language model to decide if a given query requires a creative response,
    such as making suggestions or composing ideas, or a technical response, like providing product details or prices.

    Returns:
    - str: The label 'creative' if the query requires creative input, or 'technical' if it requires technical information.
    """
    
   
    PROMPT = f"""Decide if the following query is a query that requires creativity (creating, composing, making new things) or technical (information about products, prices etc.). Label it as creative or technical.
        Examples:
        Give me suggestions on a nice look for a nightclub. Label: creative
        What are the blue dresses you have available? Label: technical
        Give me three Tshirts for summer. Label: technical
        Give me a look for attending a wedding party. Label: creative
        Give me suggestions on clothes that match a green Tshirt. Label: creative
        I would like a suggestion on which products match a green Tshirt I already have. Label: creative

        Query to be analyzed: {query}. Only output one token with the label
        """


    kwargs = generate_params_dict(PROMPT, temperature = 0, max_tokens = 1)

    response = generate_with_single_input(**kwargs) 

    # Get the Label by accessing the content key of the response dictionary
    label = response['choices'][0]['message']['content']
    total_tokens = response['usage']['total_tokens']

    return label, total_tokens

In [311]:
queries = ["Give me two sneakers with vibrant colors.",
           "What are the most expensive clothes you have in your catalogue?",
           "I have a green Dress and I like a suggestion on an accessory to match with it.",
           "Give me three trousers with vibrant colors you have in your catalogue.",
           "Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather."
           ]

labels = ['technical', 'technical', 'creative', 'technical', 'creative']

In [312]:
for query, correct_label in zip(queries, labels):
    response, total_tokens = decide_task_nature(query)
    label = response
    if label == correct_label:
        label = "\033[32m" + label + "\033[0m" 
    else:
        label = "\033[31m" + label + "\033[0m"
    if total_tokens > 170:
        total_tokens = "\033[31m"  + str(total_tokens) + "\033[0m"
    else:
        total_tokens = "\033[32m"  + str(total_tokens) + "\033[0m"
    print(f"Query: {query} Label Predicted: {label}. Correct Label: {correct_label} Total Tokens: {total_tokens}")

Query: Give me two sneakers with vibrant colors. Label Predicted: technical. Correct Label: technical Total Tokens: 196
Query: What are the most expensive clothes you have in your catalogue? Label Predicted: technical. Correct Label: technical Total Tokens: 200
Query: I have a green Dress and I like a suggestion on an accessory to match with it. Label Predicted: creative. Correct Label: creative Total Tokens: 206
Query: Give me three trousers with vibrant colors you have in your catalogue. Label Predicted: technical. Correct Label: technical Total Tokens: 201
Query: Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather. Label Predicted: creative. Correct Label: creative Total Tokens: 212


### 📦 Start: **Products** section

### Clasify Query on creative vs technical tasks

Returns a small dict of `top_p` and `temperature` tuned for the task: higher randomness for **creative**, lower for **technical**, with a sensible default fallback.


In [313]:
def get_params_for_task(task):
    """
    Retrieves specific language model parameters based on the task nature.

    This function provides parameter sets tailored for creative or technical tasks to optimize
    language model behavior. For creative tasks, higher randomness is encouraged, while technical
    tasks are handled with more focus and precision. A default parameter set is provided for unexpected cases.

    Parameters:
    - task (str): The nature of the task ('creative' or 'technical').

    Returns:
    - dict: A dictionary containing 'top_p' and 'temperature' settings for the specified task.
    """
    # Create the parameters dict for technical and creative tasks
    PARAMETERS_DICT = {"creative": {'top_p': 0.9, 'temperature': 1},
                       "technical": {'top_p': 0.7, 'temperature': 0.3}} 
    
    # If task is technical, return the value for the key technical in PARAMETERS_DICT
    if task == 'technical':
        param_dict = PARAMETERS_DICT['technical'] 

    # If task is creative, return the value for the key creative in PARAMETERS_DICT
    elif task == 'creative':
        param_dict = PARAMETERS_DICT['creative'] 

    # If task is a different value, fallback to another set of parameters
    else: 
        param_dict = {'top_p': 0.5, 'temperature': 1} 

    
    return param_dict

In [314]:
# Let's remember the data structure of a product
products_data[0]

{'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Shirts',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011,
 'usage': 'Casual',
 'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
 'price': 67.0,
 'product_id': 15970}

This is a dictionary with every possible value for the categories the LLM can pick from to generate a JSON.

In [315]:
# Run this cell to generate the dictionary with the possible values for each key
values = {}
for d in products_data:
    for key, val in d.items():
        if key in ('product_id', 'price', 'productDisplayName', 'subCategory', 'year'):
            continue
        if key not in values.keys():
            values[key] = set()
        values[key].add(val)

In [316]:
# Example of possible values for the feature 'season'
values['season']

{'All seasons', 'Fall', 'Spring', 'Summer', 'Winter'}

### Generate metadata JSON for product filtering

Prompts the LLM (low temperature) to produce a **strict JSON** describing filters (gender, masterCategory, articleType, baseColour, price, usage, season) using only allowed values from `values`. Returns the JSON **string** plus `total_tokens`; you can then parse/validate it and apply as filters in the `Products` collection.


In [317]:
def generate_metadata_from_query(query):
    """
    Generates metadata in JSON format based on a given query to filter clothing items.

    This function constructs a prompt for a language model to create a JSON object that will
    guide the filtering of a vector database query for clothing items. It takes possible values from
    a predefined set and ensures only relevant metadata is included in the output JSON.

    Parameters:
    - query (str): The query describing specific clothing-related needs.

    Returns:
    - str: A JSON string representing metadata with keys like gender, masterCategory, articleType,
      baseColour, price, usage, and season. Each value in the JSON is within a list, with prices specified
      as a dict containing "min" and "max" values. Unrestricted keys should use ["Any"] and unspecified
      prices should default to {"min": 0, "max": "inf"}.
    """

    PROMPT = f"""
    One query will be provided. For the given query, there will be a call on vector database to query relevant clothing items. 
    Generate a JSON with useful metadata to filter the products in the query. Possible values for each feature is in the following json: 
    {values}

    Provide a JSON with the features that best fit in the query (can be more than one, write in a list). Also, if present, add a price key, saying if there is a price range (between values, greater than or smaller than some value).
    Only return the JSON, nothing more. price key must be a JSON with "min" and "max" values (0 if no lower bound and inf if no upper bound). 
    Always include gender, masterCategory, articleType, baseColour, price, usage and season as keys. All values must be within lists.
    If there is no price set, add min = 0 and max = inf.
    Only include values that are given in the json above. 
    
    Example of expected JSON:

    {{
    "gender": ["Women"],
    "masterCategory": ["Apparel"],
    "articleType": ["Dresses"],
    "baseColour": ["Blue"],
    "price": {{"min": 0, "max": "inf"}},
    "usage": ["Formal"],
    "season": ["All seasons"]
    }}

    Query: {query}
             """
    
    # Generate the response with the generate_with_single_input, PROMPT, temperature = 0 (low randomness) and max_tokens = 1500.
    kwargs = {"prompt": PROMPT, 'temperature': 0, "max_tokens": 1500} 

    response = generate_with_single_input(**kwargs) 

    # Get the Label by accessing the content key of the response dictionary
    content = response['choices'][0]['message']['content']
    total_tokens = response['usage']['total_tokens']
 

    
    return content, total_tokens

In [318]:
content, total_tokens = generate_metadata_from_query("Create a look for a man that suits a sunny day in the park. I don't want to spend more than 300 dollars on each piece.")
print(content)
print(total_tokens)

{
    "gender": ["Men"],
    "masterCategory": ["Apparel"],
    "articleType": ["Shirts", "Shorts"],
    "baseColour": ["Yellow", "Orange", "Green", "Blue", "White"],
    "price": {"min": 0, "max": 300},
    "usage": ["Casual", "Smart Casual"],
    "season": ["Summer"]
}
1460


### Create a vectorized **Products** collection

Drops any existing `Products` class and recreates it with the `text2vec-transformers` vectorizer so `near_text` works out of the box. Defines core product attributes (text fields + numeric `year`, `price`, and `product_id`) and returns a handle to the collection.


In [319]:
"""
# Drop & recreate to ensure it has a vectorizer
try:
    if client.collections.exists("Products"):
        client.collections.delete("Products")
except Exception as e:
    print("Warning while deleting Products:", e)

client.collections.create(
    name="Products",
    vectorizer_config=Configure.Vectorizer.text2vec_transformers(),  # <-- key for near_text
    properties=[
        Property(name="productDisplayName", data_type=DataType.TEXT),
        Property(name="articleType",        data_type=DataType.TEXT),
        Property(name="baseColour",         data_type=DataType.TEXT),
        Property(name="gender",             data_type=DataType.TEXT),
        Property(name="season",             data_type=DataType.TEXT),
        Property(name="usage",              data_type=DataType.TEXT),
        Property(name="masterCategory",     data_type=DataType.TEXT),
        Property(name="subCategory",        data_type=DataType.TEXT),
        Property(name="year",               data_type=DataType.INT),
        Property(name="price",              data_type=DataType.NUMBER),
        Property(name="product_id",         data_type=DataType.INT),
    ],
)

products_collection = client.collections.get("Products")
"""


'\n# Drop & recreate to ensure it has a vectorizer\ntry:\n    if client.collections.exists("Products"):\n        client.collections.delete("Products")\nexcept Exception as e:\n    print("Warning while deleting Products:", e)\n\nclient.collections.create(\n    name="Products",\n    vectorizer_config=Configure.Vectorizer.text2vec_transformers(),  # <-- key for near_text\n    properties=[\n        Property(name="productDisplayName", data_type=DataType.TEXT),\n        Property(name="articleType",        data_type=DataType.TEXT),\n        Property(name="baseColour",         data_type=DataType.TEXT),\n        Property(name="gender",             data_type=DataType.TEXT),\n        Property(name="season",             data_type=DataType.TEXT),\n        Property(name="usage",              data_type=DataType.TEXT),\n        Property(name="masterCategory",     data_type=DataType.TEXT),\n        Property(name="subCategory",        data_type=DataType.TEXT),\n        Property(name="year",             

### Batch-ingest cleaned `products_data` with stable UUIDs

Cleans each product (safe casting for `year/price/product_id`), then batch-inserts into the vectorized `Products` collection with a fixed batch size and concurrency. UUIDs are derived from `product_id` (fallback to name) for idempotent re-runs. Weaviate’s `text2vec-transformers` auto-vectorizes on insert, enabling `near_text` queries later.


In [320]:
from tqdm import tqdm
import math

def _to_int(x):
    try:
        if x is None or (isinstance(x, float) and math.isnan(x)) or (isinstance(x, str) and not x.strip()):
            return None
        return int(float(x))
    except Exception:
        return None

def _to_float(x):
    try:
        if x is None or (isinstance(x, float) and math.isnan(x)) or (isinstance(x, str) and not x.strip()):
            return None
        return float(x)
    except Exception:
        return None

def _clean_obj(d):
    return {
        "productDisplayName": d.get("productDisplayName"),
        "articleType":        d.get("articleType"),
        "baseColour":         d.get("baseColour"),
        "gender":             d.get("gender"),
        "season":             d.get("season"),
        "usage":              d.get("usage"),
        "masterCategory":     d.get("masterCategory"),
        "subCategory":        d.get("subCategory"),
        "year":               _to_int(d.get("year")),
        "price":              _to_float(d.get("price")),
        "product_id":         _to_int(d.get("product_id")),
    }


with products_collection.batch.fixed_size(batch_size=200, concurrent_requests=4) as batch:
    for d in tqdm(products_data):
        obj = _clean_obj(d)
        # use a stable UUID; product_id preferred, else fallback to name
        key = str(obj.get("product_id") or obj.get("productDisplayName") or repr(obj))
        batch.add_object(properties=obj, uuid=generate_uuid5(key))

100%|██████████| 44424/44424 [00:03<00:00, 12989.53it/s]


In [321]:
len(products_collection)

44424

<a id='5-3'></a>

<a id='5-3'></a>
### 5.3 Filtering by Metadata

The functions used to filter by metadata have been moved to the **`utils.py`** file.
You can find this file in the **File Browser** on the left panel.

You worked with these functions in the previous assignment, but for this one, **you won’t need to use them directly**.

So, let’s go ahead and jump into the exercise!

### Semantic product search (top-20)

Queries the vectorized `Products` collection with `near_text` using the user’s `query`, returning the top 20 matched objects. No LLM is called here, so token usage is `0`. 



In [322]:
def get_relevant_products_from_query(query):
    """
    Retrieve the most relevant products for a given query by applying semantic search.

    Parameters:
    query (str): The query string used to search for relevant products.
    Returns:
    list: A list of product objects that are most relevant to the query.
    total_tokens: The number of tokens used in the LLM call. Returns 0 if simplified search is used.
    """

    
    #do a semantic search with 20 objects and return it           
    results = products_collection.query.near_text(f"{query}", limit=20)  
        
    return results.objects, 0  # Return the objects and 0 tokens since no LLM call was made


In [323]:
query = "Give me three T-shirts to use in sunny days"

In [324]:
t, total_tokens = get_relevant_products_from_query(query)

In [325]:
total_tokens

0

### Compile product results into a prompt-friendly context block

Takes a list of Weaviate result objects and builds a multi-line string with key attributes (ID, name, category, usage, gender, type, color, season, year) per product—handy to feed into an LLM. 


In [326]:
def generate_items_context(results):
    """
    Compile detailed product information from a list of result objects into a formatted string.

    Parameters:
    results (list): A list of result objects, each having a `properties` attribute that is a dictionary 
                    containing product attributes such as 'product_id', 'productDisplayName', 
                    'masterCategory', 'usage', 'gender', 'articleType', 'subCategory', 
                    'baseColour', 'season', and 'year'.

    Returns:
    str: A multi-line string where each line contains the formatted details of a single product.
         Each product detail includes the product ID, name, category, usage, gender, type, color, 
         season, and year.
    """
    t = ""  # Initialize an empty string to accumulate product information

    for item in results:  # Iterate through each item in the results list
        item = item.properties  # Access the properties dictionary of the current item

        # Append formatted product details to the output string
        t += (
            f"Product ID: {item['product_id']}. "
            f"Product name: {item['productDisplayName']}. "
            f"Product Category: {item['masterCategory']}. "
            f"Product usage: {item['usage']}. "
            f"Product gender: {item['gender']}. "
            f"Product Type: {item['articleType']}. "
            f"Product Category: {item['subCategory']} "
            f"Product Color: {item['baseColour']}. "
            f"Product Season: {item['season']}. "
            f"Product Year: {item['year']}.\n"
        )

    return t  # Return the complete formatted string with product details

In [327]:
print(generate_items_context(t)[:1000])

Product ID: 35885. Product name: Jungle Book Boys Yee-Ha! White T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Boys. Product Type: Tshirts. Product Category: Topwear Product Color: White. Product Season: Summer. Product Year: 2012.
Product ID: 36325. Product name: Mr.Men Boys Mr. Happy White T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Boys. Product Type: Tshirts. Product Category: Topwear Product Color: White. Product Season: Summer. Product Year: 2012.
Product ID: 36324. Product name: Mr.Men Boys Mr. Happy White T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Boys. Product Type: Tshirts. Product Category: Topwear Product Color: White. Product Season: Summer. Product Year: 2012.
Product ID: 12226. Product name: Basics Men Pack Of 3 T-shirts. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Tshirts. Product Category: Topwear Product Color: White. Product Season: Summ

### Build LLM prompt for product answers (creative/technical routing)

Classifies the query (creative vs technical), sets decoding params, retrieves top-20 relevant products via semantic search, composes a compact product-context block, and returns `kwargs` (prompt + params) for the LLM along with total token count used for routing.


In [328]:
def query_on_products(query):
    """
    Execute a product query process to generate a response based on the nature of the query.

    Parameters:
    query (str): The input query string that needs to be analyzed and answered using product data.

    Returns:
    dict: A dictionary of keyword arguments (`kwargs`) containing the prompt and additional settings 
          for creating a response, suitable for input to an LLM or other processing system.
    int: Number of tokens used in the process to create the kwargs dictionary


    """
    total_tokens = 0
    # Determine if the query is technical or creative in nature
    
    query_label, tokens = decide_task_nature(query)
    
    # Sum the tokens used to decide the task nature (creative or technical)
    total_tokens += tokens

    # Obtain necessary parameters based on the query type
    parameters_dict = get_params_for_task(query_label)
    
    # Retrieve products that are relevant to the query
    relevant_products, tokens = get_relevant_products_from_query(query)
    
    # Sum the tokens used to get relevant products 
    total_tokens += tokens
     
    # Create a context string from the relevant products
    context = generate_items_context(relevant_products)

    # Construct a prompt including product details and the query. Remember to add the context and the query in the prompt, also, ask the LLM to provide the product ID in the answer
    PROMPT = (
        f"Given the available set of clothing products given by: "
        f"CLOTHING PRODUCTS AVAILABLE:\n{context}\n"
        f"Answer the question that follows.\n"
        f"Never use more than 5 clothing products available below to compose your answer.\n"
        f"Provide the item ID in your answers.\n"
        f"The other information might be provided but not necessarily all of them, pick only the relevant ones for the given query.\n"
        
        f"QUERY: {query}"
    )
    
    # Generate kwargs (parameters dict) for parameterized input to the LLM with , Prompt, role = 'assistant' and **parameters_dict
    kwargs = generate_params_dict(PROMPT, role='assistant', **parameters_dict) 

    
    return kwargs, total_tokens

In [329]:
kwargs, total_tokens = query_on_products('Make a wonderful look for a man attending a wedding party happening during night.')

In [330]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

To create a wonderful look for a man attending a wedding party during the night, I recommend combining the following clothing products:

1. Product ID: 28365. IDEE Men Brown Sunglasses - Although it's summer, wearing sunglasses can add a touch of sophistication and elegance to a man's outfit. The brown color will complement a variety of colors and won't distract from the overall look.
2. Product ID: 25066. Lino Perros Men Formal Turquoise Blue Accessory Gift Set - This gift set includes a tie, pocket square, and cufflinks, which can add a pop of color and personality to the outfit. The turquoise blue color will add a unique touch to a classic wedding party attire.
3. Product ID: 49696. Park Avenue Red Checked Tie - A red tie can be a bold and eye-catching choice for a wedding party. The checked pattern will add texture and visual interest to the outfit.
4. Product ID: 31186. Cabarelli Men Accessory Gift Set (with black color) - A simple black gift set can provide a versatile and classi

In [331]:
print(f"Total tokens used in the query is: {total_tokens + result['usage']['total_tokens']}")

Total tokens used in the query is: 1900


### Route query (FAQ vs Product) and prepare final LLM kwargs

Classifies the user query, calls the matching pipeline (`query_on_faq` or `query_on_products`), accumulates token usage, and sets the final `model` before returning `(kwargs, total_tokens)`. Includes fallbacks for undefined labels and product-pipeline errors.


In [332]:
def answer_query(query, model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",simplified=False):
    """
    Processes a user's query to determine its type (FAQ or Product) and executes the appropriate workflow.
    
    Parameters:
    - query (str): The query string provided by the user.
    - model (str): The model that will answer the question. Defaults to meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'
    - simplified (bool): If True, uses a simplified version of the method. Defaults to False.
    
    Returns:
    - dict: A dictionary containing keyword arguments for further processing.
      If the query is neither FAQ nor Product-related, returns a default response dictionary instructing
      the assistant to answer based on existing context.
    """
    # Initialize the total tokens used to zero
    total_tokens = 0
    
    # Determine if the query is FAQ or Product and get the token count for this step
    label, tokens = check_if_faq_or_product(query)
    
    # Sum the tokens
    total_tokens += tokens
    
    # If the query is neither FAQ nor Product, return a default response
    if label not in ['FAQ', 'Product']:
        return {
            "role": "assistant",
            "prompt": (f"User provided a question that does not fit FAQ or Product-related categories. "
                       f"Answer it based on the context you already have. Query provided by the user: {query}")
        }
    
    # Process the query based on its label
    if label == 'FAQ':
        # Handle FAQ-related queries
        kwargs = query_on_faq(query)
    elif label == 'Product':
        try:
            # Handle Product-related queries, with error handling in place
            kwargs, tokens = query_on_products(query)
            # Add the tokens to the total tokens
            total_tokens += tokens
        except Exception:
            # Return an error response if an exception occurs during querying
            return {
                "role": "assistant",
                "prompt": (f"User provided a question that broke the querying system. "
                           f"Instruct them to rephrase it. Answer it based on the context you already have. "
                           f"Query provided by the user: {query}")
            }, total_tokens
    # Set the model to answer the final query - usually a better one         
    kwargs['model'] = model
    # Return the kwargs and total_tokens for further processing
    return kwargs, total_tokens

# 🏁 Final Output

Runs the end-to-end pipeline for a user query:

* Classifies the query (**FAQ** vs **Product**); if Product, also (**creative** vs **technical**).
* Pulls top products via **semantic search** from Weaviate and composes a compact context.
* Builds the final **prompt + decoding params** (`kwargs`) and reports `total_tokens`.




In [333]:
Client_Query = "Give me three examples of blue t-shirts available on your catalogue."

In [334]:
kwargs, total_tokens = answer_query(Client_Query)

In [335]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

Here are three examples of blue t-shirts available on the catalogue:

1. Product ID: 21234. Product name: Basics Men Blue Printed T-shirt. Product Color: Blue. Product Season: Summer. Product Year: 2012.
2. Product ID: 8317. Product name: Wildcraft Men Blue Printed T-shirt. Product Color: Blue. Product Season: Summer. Product Year: 2012.
3. Product ID: 12211. Product name: Basics Men Blue Printed T-shirt. Product Color: Blue. Product Season: Fall. Product Year: 2011.


In [336]:
# To get the total tokens for the call, we must sum the total_tokens to get the kwargs dictionary + total tokens from the LLM call
total_tokens +  result['usage']['total_tokens']

1951